In [14]:
import keras
import matplotlib.pyplot as plt
from IPython.display import clear_output

class PlotLosses(keras.callbacks.Callback):
    def __init__(self, plot_interval=1, evaluate_interval=10, x_val=None, y_val_categorical=None):
        self.plot_interval = plot_interval
        self.evaluate_interval = evaluate_interval
        self.x_val = x_val
        self.y_val_categorical = y_val_categorical
        #self.model = model
    
    def on_train_begin(self, logs={}):
        print('Begin training')
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.logs = []
    
    def on_epoch_end(self, epoch, logs={}):
        if self.evaluate_interval is None:
            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.acc.append(logs.get('acc'))
            self.val_acc.append(logs.get('val_acc'))
            self.i += 1
        
        if (epoch%self.plot_interval==0):
            clear_output(wait=True)
            f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(20,5))
            ax1.plot(self.x, self.losses, label="loss")
            ax1.plot(self.x, self.val_losses, label="val_loss")
            ax1.legend()

            ax2.plot(self.x, self.acc, label="acc")
            ax2.plot(self.x, self.val_acc, label="val_acc")
            ax2.legend()
            plt.show();
        #score = self.model.evaluate(x_test, y_test_categorical, verbose=0)
        
        #print("accuracy: ", score[1])
    
    def on_batch_end(self, batch, logs={}):
        if self.evaluate_interval is not None:
            if (batch%self.evaluate_interval==0):
                self.i += 1
                self.logs.append(logs)
                self.x.append(self.i)
                self.losses.append(logs.get('loss'))
                self.acc.append(logs.get('acc'))

                if self.x_val is not None:
                    score = self.model.evaluate(self.x_val, self.y_val_categorical, verbose=0)
                    self.val_losses.append(score[0])
                    self.val_acc.append(score[1])

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

# Cargo datos

In [16]:
folder = 'C:/Users/Lenovo/Documents/GitHub/Datasets/Kaggle - Imagenes - MNIST/'
x = np.load(folder+'train_images.npy')
y = np.loadtxt(folder+'train_labels.csv', delimiter=',', skiprows=1)
x_test = np.load(folder+'test_images.npy')

In [17]:
print("x shape:{}".format(x.shape))
print("y shape:{}".format(y.shape))
print("x_test shape:{}".format(x_test.shape))

x shape:(60000, 28, 28)
y shape:(60000,)
x_test shape:(10000, 28, 28)


In [18]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1)

In [19]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization


## Reshape del dataset para usarlo en la CNN (Falta el Depth)

In [20]:
#Rows and columns from images.
img_rows = 28
img_cols = 28
#Reshape del train y valid sets.
x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_valid = x_valid.reshape(x_valid.shape[0],img_rows,img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
#Defino el imput_shape
input_shape = (img_rows,img_cols,1)

In [21]:
print("x_train shape:{}".format(x_train.shape))
print("x_valid shape:{}".format(x_valid.shape))

x_train shape:(54000, 28, 28, 1)
x_valid shape:(6000, 28, 28, 1)


In [22]:
y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_valid)

# Red neuronal - CNN

In [23]:
p = 0.25 #Dropout

# Creo el modelo
model=Sequential()

#model.add(Convolution2D(filters=16, kernel_size=(3, 3), padding="same", name='Conv1',input_shape=input_shape))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv1',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(p))

#model.add(Convolution2D(filters=16, kernel_size=(3, 3), padding="same", name='Conv2',input_shape=input_shape))
model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding="same", name='Conv2',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(p))

model.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool1'))
model.add(Dropout(p))

"""
#model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv3',input_shape=input_shape))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv3',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

#model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv4',input_shape=input_shape))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv4',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool2'))
model.add(Dropout(p))
"""
model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
MaxPool1 (MaxPooling2D)      (None, 14, 14, 64)        0         
__________

In [24]:
# Compilo el modelo
lr = 0.0001 
#lr = 0.1

ADAM = optimizers.Adam(lr=lr)

model.compile(loss = 'categorical_crossentropy', optimizer=ADAM, metrics=['accuracy'])

In [25]:
# Callbacks

## Callback para graficar
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=60, x_val=x_valid, y_val_categorical=y_val_categorical)
## Callback para guardar pesos
checkpointer = ModelCheckpoint(filepath='cnn.mnist.hdf5', verbose=1, save_best_only=True)

In [26]:
batch_size = 1024
epochs = 10

model.fit(x_train
          ,y_train_categorical
          ,epochs=epochs
          ,validation_data = (x_valid, y_val_categorical)
          ,callbacks=[plot_losses, checkpointer],)

Train on 54000 samples, validate on 6000 samples
Begin training
Epoch 1/10
   32/54000 [..............................] - ETA: 49:36 - loss: 2.8059 - acc: 0.0000e+00

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (37.968916). Check your callbacks.
  % delta_t_median)


   64/54000 [..............................] - ETA: 9:28:11 - loss: 2.7120 - acc: 0.0625  

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (18.984458). Check your callbacks.
  % delta_t_median)


 9632/54000 [====>.........................] - ETA: 31:02 - loss: 0.5836 - acc: 0.7939

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(x_valid, y_val_categorical)

In [ ]:
print(acc)

In [ ]:
test_prediction = model.predict(x_test)

In [ ]:
test_labels = np.argmax(test_prediction, axis = 1)

In [ ]:
import pandas
df = pandas.DataFrame(data={"Category": test_labels}).astype(int)
df.to_csv("C:/Users/Lenovo/Documents/GitHub/Lab_ML_ITBA/multiclass/submission_cnn.csv", sep=',',index=True,  index_label='Id')